In [1]:
# import packages here to reduce the size of code cells later

import pandas as pd
from prettypandas import PrettyPandas
import patsy

import numpy as np
import scipy.stats
import statsmodels.formula.api
import statsmodels.api as sm

from graphviz import Digraph
import seaborn as sns

import dexpy.factorial
import dexpy.alias
import dexpy.power

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import patches

from IPython.display import display, Markdown, HTML


ModuleNotFoundError: No module named 'prettypandas'

In [ ]:
# helper functions for this notebook

# use SVG for matplotlib-based figures
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

def coded_to_actual(coded_data, actual_lows, actual_highs):
    """Converts a pandas DataFrame from coded units to actuals."""
    actual_data = coded_data.copy()
    for col in actual_data.columns:
        if not (col in actual_highs and col in actual_lows):
            continue
        try:
            # convert continuous variables to their actual value
            actual_data[col] *= 0.5 * (float(actual_highs[col]) - float(actual_lows[col]))
            # don't need to cast to float here, if either are not a float exception will have been thrown
            actual_data[col] += 0.5 * (actual_highs[col] + actual_lows[col])
        except ValueError:
            # assume 2 level categorical
            actual_data[col] = actual_data[col].map({-1: actual_lows[col], 1: actual_highs[col]})
    return actual_data
        
def get_tick_labels(key, lows, highs, units):
    """Returns a list of low/high labels with units (e.g. [8mm, 10mm])"""
    return [str(lows[key]) + units[key], str(highs[key]) + units[key]]

# Design of Experiments in Python

Hank Anderson  
<hank@statease.com>  
https://github.com/hpanderson/dexpy-pymntos  

# Agenda

* What is Design of Experiments?
* Design an experiment with `dexpy` to improve the office coffee
* Measure the power of our experiment
* Model the taste test results using `statsmodels`
* Visualize the data with `seaborn` and `matplotlib` to find the best pot of coffee

# What is Design of Experiments (DoE)?

A systematic series of tests, in which purposeful changes are made to input factors, so that you may identify causes for significant changes in the output responses.

NIST has a [nice primer on DOE](http://www.itl.nist.gov/div898/handbook/pri/section1/pri1.htm).

In [ ]:
dot = Digraph(comment='Design of Experiments')
dot.body.extend(['rankdir=LR', 'size="10,10"'])
dot.node_attr.update(shape='rectangle', style='filled', fontsize='20', fontname="helvetica")

dot.node('X', 'Controllable Factors', color='mediumseagreen', width='3')
dot.node('Z', 'Noise Factors', color='indianred2', width='3')
dot.node('P', 'Process', color='lightblue', height='1.25', width='3')
dot.node('Y', 'Responses', color='lightblue')

dot.edges(['XP', 'ZP', 'PY'] * 3)

dot

# What Is It For?

* Screening Experiments - Determining which inputs to your system are important
* Modeling a Process - Gain a better understanding of your system
* Optimization - Find the combination of inputs that results in a better output
* Robustness - Find a combination of inputs that produces a consistent result

# DoE in Python: dexpy

* Based on Design-Expert<sup>&reg;</sup> software, a package for design and analysis of industrial experiments
* Apache2 licensed, pure python (for now), available on pypi
* Requires `numpy`, `scipy`, `pandas` and `patsy`
* Recommend `statsmodels` for analysis, `matplotlib` and `seaborn` for visualization
* Other alternatives are:
 * [pyDOE](https://github.com/tisimst/pyDOE) (not actively maintained)
 * R [???](https://cran.r-project.org/web/views/ExperimentalDesign.html)
 * [SciDOE](https://forge.scilab.org/index.php/p/scidoe/)
 * [gosset](http://neilsloane.com/gosset/)
 * Proprietary software ($$$) - [Design-Expert](https://www.statease.com), [JMP](https://www.jmp.com), [Minitab](https://www.minitab.com)

# Motivating Example: Better Office Coffee

 * Current coffee is subpar ("disgusting and unacceptable")
 * Need to answer the following questions via experimentation:
  * What coffee wt to use?
  * How much coffee to use?
  * How to grind the coffee?

# Motivating Example: Better Office Coffee

 * 5 input factors
  * lh of Coffee (2.5 to 4.0 oz.)
  * Grind size (8-10mm)
  * Brew time (3.5 to 4.5 minutes)
  * Grind Type (burr vs blade)
  * Coffee wt (light vs dark)
 * 1 response: Average overall liking by a panel of 5 office coffee addicts
  * Each taster rates the coffee from 1-9
 * Maximum of 3 taste tests a day, for liability reasons
 

In [ ]:
# Layer Thickness = lh
lh_low = .16 
lh_hi = .24 

# Print Speed = ps
ps_low = 50 
ps_hi = 60 

# Infill Density = id
id_low = .25 
id_hi = .15 

# Raster Width = rw
rw_low = .4 
rw_hi = .8 

# Wall Thicknessv = wt
wt_low = 1.2 
wt_hi = .8

In [ ]:
# set some variables related to the coffee data set
actual_lows = { 'lh' : lh_low, 'ps' : ps_low, 'id': id_low, 'rw': rw_low, 'wt': wt_low }
actual_highs = { 'lh' : lh_hi, 'ps' : ps_hi, 'id': id_hi, 'rw': rw_hi, 'wt': wt_hi }
units = { 'lh' : 'mm', 'ps' : 'mm/s', 'id': '%', 'rw': 'mm', 'wt': 'mm' }

# One Factor at a Time Experimentation

* Hold all inputs constant, then change one at a time
* Cannot detect interactions, e.g. grind size and lh of coffee have a synergistic relationship
* Inefficient

In [ ]:
points = [
    [-1, -1],
    [-1, 1],
    [1, -1],
    [-1, -1],
    [-1, 1],
    [1, -1],
]
df = pd.DataFrame(points, columns=['ps', 'lh'])
fg = sns.lmplot('lh', 'ps', data=df, fit_reg=False)

p = patches.Polygon(points, color="navy", alpha=0.3, lw=2)
ax = fg.axes[0, 0]
ax.add_patch(p)
ax.set_xticks([-1, 1])
ax.set_xticklabels(get_tick_labels('lh', actual_lows, actual_highs, units))
ax.set_yticks([-1, 1])
ax.set_yticklabels(get_tick_labels('ps', actual_lows, actual_highs, units))

p = patches.Polygon([[-1, 1], [1, 1], [1, -1]], color="firebrick", alpha=0.3, lw=2)
p = ax.add_patch(p)


In [ ]:
cube_design = dexpy.factorial.build_factorial(3, 8)

points = np.array(cube_design)
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d', facecolor='w')
ax.view_init(30, -60) # rotate plot

X, Y = np.meshgrid([-1,1], [-1,1])

cube_alpha = 0.1
ax.plot_surface(X, Y, np.atleast_2d(1), alpha=cube_alpha, color="r")
ax.plot_surface(X, Y, np.atleast_2d(-1), alpha=cube_alpha)
ax.plot_surface(X, -1, Y, alpha=cube_alpha)
ax.plot_surface(X, 1, Y, alpha=cube_alpha, color="r")
ax.plot_surface(1, X, Y, alpha=cube_alpha, color="r")
ax.plot_surface(-1, X, Y, alpha=cube_alpha)
ax.scatter3D(points[:, 0], points[:, 1], points[:, 2],
             c=["b", "b", "b", "r", "b", "r", "r", "r"])

ax.set_xticks([-1, 1])
ax.set_xticklabels(get_tick_labels('ps', actual_lows, actual_highs, units))
ax.set_yticks([-1, 1])
ax.set_yticklabels(get_tick_labels('lh', actual_lows, actual_highs, units))
ax.set_zticks([-1, 1])
ax.set_zticklabels(get_tick_labels('wt', actual_lows, actual_highs, units))

plt.show()

# Factorial Design

* Change multiple inputs at once
* Reveals interactions
* Maximizes information with minimum runs

In [ ]:
df = dexpy.factorial.build_factorial(2, 4)
df.columns = ['lh', 'ps']

fg = sns.lmplot('lh', 'ps', data=df, fit_reg=False)
ax = fg.axes[0, 0]
ax.set_xticks([-1, 1])
ax.set_xticklabels(get_tick_labels('lh', actual_lows, actual_highs, units))
ax.set_yticks([-1, 1])
ax.set_yticklabels(get_tick_labels('ps', actual_lows, actual_highs, units))
p = ax.add_patch(patches.Rectangle((-1, -1), 2, 2, color="navy", alpha=0.3, lw=2))

In [ ]:
cube_design = dexpy.factorial.build_factorial(3, 8)

points = np.array(cube_design)
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d', facecolor='w')
ax.view_init(30, -60) # rotate plot

X, Y = np.meshgrid([-1,1], [-1,1])

cube_alpha = 0.1
ax.plot_surface(X, Y, np.atleast_2d(1), alpha=cube_alpha)
ax.plot_surface(X, Y, np.atleast_2d(-1), alpha=cube_alpha)
ax.plot_surface(X, -1, Y, alpha=cube_alpha)
ax.plot_surface(X, 1, Y, alpha=cube_alpha)
ax.plot_surface(1, X, Y, alpha=cube_alpha)
ax.plot_surface(-1, X, Y, alpha=cube_alpha)
ax.scatter3D(points[:, 0], points[:, 1], points[:, 2], c="b")

ax.set_xticks([-1, 1])
ax.set_xticklabels(["50mm", "60mm"])
ax.set_yticks([-1, 1])
ax.set_yticklabels([".15", ".2"])
ax.set_zticks([-1, 1])
ax.set_zticklabels(["1.2mm", ".8mm"])

plt.show()

# Statistical Power

The probability that a design will detect an active effect.

| Effect?       | Retain H<sub>0</sub>   | Reject H<sub>0</sub>    |
| ------------- |:-----------------------| -----------------------:|
| **No**        | OK                     | Type 1 Error            |
| **Yes**       | Type II Error          | OK                      |

Power is expressed as a probability to detect an effect of size Δ, given noise σ. This is typically given as a delta to sigma ratio Δ/σ. Power is a function of the signal to noise ratio, as well as the number and layout of experiments in the design.

In [ ]:
runs = 50
delta = 0.5
sigma = 1.5
alpha = 0.05

one_factor = pd.DataFrame([ -1, 1 ] * runs, columns=['wt'])
one_factor_power = dexpy.power.f_power('wt', one_factor, delta/sigma, alpha)

display(Markdown('''
# Power Example
{} pots of coffee are tested with light wt, then {} pots with dark wt.
There is a variance of {} taste rating from pot to pot. If we expect a {} change
in the taste rating when going from light to dark, what is the likelihood we would detect it?
(Answer: **{:.2f}%**)

Note: this assumes that we reject H<sub>0</sub> at p <= {}
'''.format(int(runs / 2), int(runs / 2), sigma, delta, one_factor_power[1]*100, alpha)
))

def plot_shift(runs, delta, sigma, annotate=False):
    """Plots two sets of random normal data, one shifted up delta units."""
    mean = 5
    low = sigma*np.random.randn(int(runs/2),1)+mean
    high = sigma*np.random.randn(int(runs/2),1)+mean+delta
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_ylabel("taste")
    ax.set_xlabel("runs")
    ax.set_ylim([0, 11])
    
    plt.plot(np.concatenate([low, high]))
    plt.plot([0, (runs/2)], [low.mean()] * 2, color='firebrick', lw=2)
    plt.plot([(runs/2), runs-1], [high.mean()] * 2, color='g', lw=2)
    
    p_value = scipy.stats.f_oneway(low, high).pvalue[0]
    if annotate:
        plt.annotate("p: {:.5f}".format(p_value),
                     xy=(runs / 2, (low.mean() + high.mean()) / 2), xycoords='data',
                     xytext=(.8, .9), textcoords='axes fraction',
                     bbox=dict(boxstyle="round4", fc="w"),
                     arrowprops=dict(arrowstyle='-[', linewidth=2, color='black', connectionstyle="angle3")
                     )

    plt.show()
    return [low, high]

low, high = plot_shift(runs, delta, sigma)

In [ ]:
increased_delta = delta*4
increased_delta_power = dexpy.power.f_power('wt', one_factor, increased_delta/sigma, alpha)

display(Markdown('''
# Power Example - Increase Delta
What if we don't care about a taste increase of 0.5? That's not that much better
than the current coffee, after all. Instead, if we say we only care about a change
in rating above {}, what is the likelihood we would detect such a change?
(Answer: **{:.5f}%**)
'''.format(increased_delta, increased_delta_power[1]*100)
))

_ = plot_shift(runs, increased_delta, sigma)

In [ ]:
decreased_sigma = sigma*0.5
decreased_sigma_power = dexpy.power.f_power('wt', one_factor, delta/decreased_sigma, alpha)

display(Markdown('''
# Power Example - Decrease Noise
Instead of lowering our standards for our noisy taste ratings, instead
we could bring in expert testers who have a much more accurate palate.
If we assume a decrease in noise from {} to {}, then we can detect a
change in rating of {} with {:.2f}% probability.
'''.format(sigma, decreased_sigma, delta, decreased_sigma_power[1]*100)
))

_ = plot_shift(runs, delta, sigma*0.1)

In [ ]:
increased_runs = runs * 4
one_factor = pd.DataFrame([ -1, 1 ] * increased_runs, columns=['wt'])
increased_runs_power = dexpy.power.f_power('wt', one_factor, delta/sigma, alpha)

display(Markdown('''
# Power Example - Increase Runs
If expert testers are too expensive, and we are unwilling to compromise
our standards, then the only remaining option is to create a more powerful
design. In this toy example, there isn't much we can do, since it's
just one factor. Increasing the runs from {} to {} gives a power of
{:.2f}%. This may be a more acceptable success rate than the original power
of {:.2f}%, however... that is a lot of coffee to drink.

For more complicated designs changing the structure of the design
can also increase power.
'''.format(runs, increased_runs, increased_runs_power[1]*100, one_factor_power[1]*100)
))
_ = plot_shift(increased_runs, delta, sigma)

# Calculating Power with dexpy

https://statease.github.io/dexpy/evaluation.html#statistical-power

In [ ]:
help(dexpy.power.f_power)

In [ ]:
base_point = [-1, -1, -1, -1, -1]
ofat_points = [base_point]

for i in range(0, 5):
    new_point = base_point[:]
    new_point[i] = 1
    ofat_points.append(new_point)

sn = 2.0
alpha = 0.05

ofat_df = pd.DataFrame(ofat_points*5, columns=['lh', 'ps', 'id', 'rw', 'wt'])
model = ' + '.join(ofat_df.columns)
ofat_power = dexpy.power.f_power('+'.join(ofat_df.columns), ofat_df, sn, alpha)
ofat_power.pop(0) # remove intercept
ofat_power = ['{0:.2f}%'.format(i*100) for i in ofat_power] # convert to %
ofat_power = pd.DataFrame(ofat_power, columns=['Power'], index=ofat_df.columns)

display(Markdown('''
# Calculating Power with dexpy: OFAT
* {} total runs, with a signal to noise ratio of 2.
* Model: `{}`
'''.format(len(ofat_df), model)))

display(PrettyPandas(ofat_power))

In [ ]:
full_design = dexpy.factorial.build_factorial(5, 2**5)
full_design.columns = ['lh', 'ps', 'id', 'rw', 'wt']
factorial_power = dexpy.power.f_power(model, full_design, sn, alpha)
factorial_power.pop(0)
factorial_power = ['{0:.2f}%'.format(i*100) for i in factorial_power] # convert to %
factorial_power = pd.DataFrame(factorial_power, columns=['Power'], index=full_design.columns)

display(Markdown('''
# Calculating Power with dexpy: Factorial
* {} total runs, with a signal to noise ratio of 2.
* Model (`patsy` for: `{}`
'''.format(len(full_design), model)))
display(PrettyPandas(factorial_power))

# Fractional Factorials

* Coffee experiment is 2<sup>5</sup> runs (32)
* We want to add 4 center point runs to check for curvature
* Total runs = 36, 3 per day if all testers are in the office
* Estimate experiment will take a month

# Fractional Factorials
* Power for the experiment is > 99%
* Full factorial is overkill
* Instead run 2<sup>5-1</sup> experiments, a "half fraction"


# Fractional Factorials in dexpy

https://statease.github.io/dexpy/design-build.html#module-dexpy.factorial

In [ ]:
help(dexpy.factorial.build_factorial)

In [ ]:


# cr6_print_history = dexpy.factorial.build_factorial(5, 2**(5-1))
# cr6_print_history.columns = ['lh', 'ps', 'id', 'rw', 'wt']
center_points = [
]

columns = ['trial', 'lh', 'ps', 'id', 'rw', 'wt', 'cost', 'time', 'quality', 'comment']

cr6_print_history = pd.read_csv('file:///Users/wilsonm/Dropbox%20(APSU%20GIS)/Projects%20(DB)/Dissertation/3DP_Experiments/Data/cr6-doe-schedule.csv', skiprows=1, names=columns)

#cr6_print_history = cr6_print_history.append(pd.DataFrame(center_points * 2, columns=cr6_print_history.columns))
cr6_print_history.index = np.arange(0, len(cr6_print_history))

display(Markdown("## 2<sup>(5-1)</sup> Factorial Design"))
display(PrettyPandas(cr6_print_history))

actual_design = coded_to_actual(cr6_print_history, actual_lows, actual_highs)
cr6_print_actual = actual_design
display(Markdown("Actual ## 2<sup>(5-1)</sup> Factorial Design"))
display(PrettyPandas(actual_design))

In [ ]:
# Transform DOE Schedule CSV dataframe to only include parameters

myDoE = cr6_print_history[['lh', 'ps', 'id', 'rw', 'wt']]
myDoE2 = cr6_print_actual[['lh', 'ps', 'id', 'rw', 'wt']]

In [ ]:
model = ' + '.join(myDoE.columns)
factorial_power = dexpy.power.f_power(model, myDoE, sn, alpha)
factorial_power.pop(0)
factorial_power = ['{0:.2f}%'.format(i*100) for i in factorial_power] # convert to %
factorial_power = pd.DataFrame(factorial_power, columns=['Power'], index=myDoE.columns)

display(Markdown('''
## 2<sup>(5-1)</sup> Factorial Power
* Power for {} total runs
* Signal to noise ratio of 2
* Model: `{}`
'''.format(len(myDoE), model)))
display(PrettyPandas(factorial_power))

In [ ]:
help(dexpy.power.f_power)

In [ ]:
twofi_model = "(" + '+'.join(myDoE.columns) + ")**2"
desc = patsy.ModelDesc.from_formula(twofi_model)
factorial_power = dexpy.power.f_power(twofi_model, myDoE, sn, alpha)


factorial_power.pop(0)
factorial_power = ['{0:.2f}%'.format(i*100) for i in factorial_power] # convert to %
factorial_power = pd.DataFrame(factorial_power, columns=['Power'], index=desc.describe().strip("~ ").split(" + "))

display(Markdown('''
## 2<sup>(5-1)</sup> Factorial Power
* Power for {} total runs
* Signal to noise ratio of 2
* Model: `{}`
'''.format(len(myDoE), twofi_model)))
display(Markdown("## Power for {} total runs, with a signal to noise ratio of 2.".format(len(myDoE))))
display(PrettyPandas(factorial_power))

In [ ]:
display(Markdown('''
# Aliasing

When you don't run all combinations of inputs, you lose the ability to estimate terms
For example, if you have three inputs the full model matrix looks like this:
'''))

three_factor_design = dexpy.factorial.build_factorial(3, 8)
X = patsy.dmatrix("(" + " + ".join(three_factor_design.columns) + ")**3", three_factor_design, return_type='dataframe')
display(PrettyPandas(X))


In [ ]:
display(Markdown('''
## Aliasing

If we remove half the runs, so that the design is 2<sup>3-1</sup>:
'''))

X.loc[[0,3,5,6]] = ''
display(PrettyPandas(X))
      
display(Markdown('''
You can see that A*B*C never changes. In addition, A = BC and B = AC.
When a term is a linear combination of another term that is called an **alias**. Aliased terms are unestimable.
'''))

## Calculating Aliases in dexpy

dexpy can calculate the terms that are linear combinations of other terms.

https://statease.github.io/dexpy/evaluation.html#alias-list

In [ ]:
help(dexpy.alias.alias_list)

### Cost Calculating Aliases in dexpy

In [ ]:
cr6_alias = cr6_print_history[['lh', 'ps', 'id', 'rw', 'wt']]

In [ ]:
display(Markdown('''
## Calculating Aliases in dexpy

Here is what that alias list looks like for our 3DP Experiment:
'''))
    
full_model = "(" + '+'.join(cr6_alias.columns) + ")**3"
aliases, _ = dexpy.alias.alias_list(full_model, cr6_alias)
for a in aliases:
    print(a)
    
display(Markdown('''
As you can see, we lose the ability to estimate some three factor interactions separately from the two factor interactions.
This is not a cause for great concern as the three factor interactions are rare in practice.
'''))

# Analysis

* [statsmodels](http://statsmodels.sourceforge.net/) has lots of routines for modeling data
* We will use [Ordinary Least Squares (OLS)](http://statsmodels.sourceforge.net/devel/examples/notebooks/generated/ols.html) to fit
* `statsmodels` typically takes `numpy` arrays for X and y data
* It also has a "formulas" api that accepts a `patsy` formula

## Cost Analysis

In [ ]:
cr6_cost_analysis = cr6_print_history[['lh', 'ps', 'id', 'rw', 'wt', 'cost']]
cr6_cost_analysis2 = cr6_print_actual[['lh', 'ps', 'id', 'rw', 'wt', 'cost']]
display(cr6_cost_analysis2)

In [ ]:
y_cost = cr6_cost_analysis['cost']
X = cr6_cost_analysis[['lh', 'ps', 'id', 'rw', 'wt']]

## An intercept is not added by default, so we need to add that here
X = sm.add_constant(X)
cost_results = sm.OLS(y_cost, X).fit()
cost_results.summary()

print(cost_results.summary())

In [ ]:
PlotWidth = 6
plt.figure(figsize=(PlotWidth, PlotWidth))
sns.regplot(x=cost_results.predict(X), y=y_cost)
plt.xlabel('Predicted %Conversion')
plt.ylabel('Actual %Conversion')
plt.title('Actual vs. Predicted %Conversion')
plt.show()

In [ ]:
display(cr6_cost_analysis)

In [ ]:
# Working Models
# quantreg
# poisson
# 

lm = statsmodels.formula.api.ols("cost ~(lh + ps + id + rw + wt)**2", data=cr6_cost_analysis).fit()
print(lm.summary())



In [ ]:
reduced_model = "lh + ps + id + rw + wt + lh:wt + ps:id + ps:rw"
lm = statsmodels.formula.api.ols("cost ~" + reduced_model, data=cr6_cost_analysis).fit()
print(lm.summary2())

# Visualization

* [seaborn](http://seaborn.pydata.org/) is built on top of `matplotlib` and adds support for `pandas` dataframes 
* Can build a plot using `seaborn`, then manipulate it with `matplotlib`
* Default themes look a lot nicer than what you get from `matplotlib` out of the box

In [ ]:
display(Markdown('''
If we take the experiment data from the design and use our new model to fit that data, then plot it against
the observed values we can get an idea for how well our model predicts. Points above the 45 degree line are
overpredicting for that combination of inputs. Points below the line predict a lower taste rating than
we actually measured during the experiment.'''))

actual_predicted = pd.DataFrame({ 'actual': cr6_print_history['cost'],
                                  'predicted': lm.fittedvalues
                                }, index=np.arange(len(cr6_cost_analysis['cost'])))
fg = sns.FacetGrid(actual_predicted, height=5)
fg.map(plt.scatter, 'actual', 'predicted')
ax = fg.axes[0, 0]
ax.plot([0, 1], [0, 1], color='g', lw=2)
ax.set_xticks(np.arange(0, 1.5))
ax.set_xlim([0, 1.])
ax.set_yticks(np.arange(0, 1))
ax.set_title('Actual vs Predicted')
_ = ax.set_ylim([0, 1])

In [ ]:
display(Markdown('''
Plotting the prediction for two factors at once shows how they interact with each other.
In this graph you can see that at the low brew time the larger grind size results in
a poor taste rating, likely because the coffee is too weak.'''))

pred_points = pd.DataFrame(1, columns = cr6_cost_analysis.columns, index=np.arange(4))
pred_points.loc[1,'ps'] = -1
pred_points.loc[3,'ps'] = -1
pred_points.loc[2,'id'] = -1
pred_points.loc[3,'id'] = -1
pred_points['cost'] = lm.predict(pred_points)
pred_points = coded_to_actual(pred_points, actual_lows, actual_highs)

fg = sns.factorplot('ps', 'cost', hue='id', kind='point', data=pred_points)
ax = fg.axes[0, 0]
ax.set_xticklabels(get_tick_labels('ps', actual_lows, actual_highs, units))
_ = ax.set_title('Print Speed/Infill Density')

In [ ]:
display(Markdown('''
This graph contains the prediction with the highest taste rating, 7.72.
However, if you look at the dark bean line there is a point where we can get
a rating of 6.93 with 2.5oz of grounds.
'''))

pred_points = pd.DataFrame(1, columns = cr6_cost_analysis.columns, index=np.arange(4))
pred_points.loc[1,'lh'] = -1
pred_points.loc[3,'lh'] = -1
pred_points.loc[2,'wt'] = -1
pred_points.loc[3,'wt'] = -1
pred_points['cost'] = lm.predict(pred_points)
pred_points = coded_to_actual(pred_points, actual_lows, actual_highs)

fg = sns.factorplot('lh', 'cost', hue='wt', kind='point', palette={'dark': 'maroon', 'light': 'peru'}, data=pred_points)
ax = fg.axes[0, 0]
ax.set_xticklabels(get_tick_labels('lh', actual_lows, actual_highs, units))
ax.set_title('lh/wt Interaction')
plt.show()

display(PrettyPandas(pred_points))
display(Markdown('''That savings of 1.5oz per pot would create a nice surplus in the coffee budget at the end of the year.'''))

![coffeemaker](img/coffee_maker.jpg)

# The End

* We were able to build and execute an experiment in Python that resulted in a better tasting (and cheaper!) coffee.
* These slides can be found at https://hpanderson.github.io/dexpy-pymntos
* The jupyter notebook they are based on can be found on my github: https://github.com/hpanderson/dexpy-pymntos
* You can reach me at: <hank@statease.com>
* The `dexpy` docs are at: https://statease.github.io/dexpy/
* `dexpy` is only at version 0.1, we plan on greatly expanding the design and analysis capabilities
 